In [1]:
import os
import pandas as pd
import json
from tqdm import tqdm

# импортируем библиотеку для работы с картами
# from keplergl import KeplerGl

# Импорт собственных классов
from ParseLog import ParseLog
from PostgreSQL import PostgreSQL

# Импорт параметров для подключения к БД
from db_parameters import *

Наименование колонок:

- `TIMESTAMP` - время в UTC в формате дата:время
- `CLID` - vologda или cherepovets
- `UUID` - уникальный идентификатор ТС в формате id_перевозчика + "_" + id_ТС_в базе_перевозчика
- `ROUTE` - "#" + номер маршрута 
- `VEHICLE_TYPE` - 'bus', 'tramway', 'minibus', 'trolleybus'
- `LATITUDE` – широта
- `LONGITUDE` – долгота
- `SPEED` - текущая скорость
- `DIRECTION` - азимут
- `PRODUCTION` - признак производственного рейса. Когда "0" - скорее всего автобус без пассажиров. В Яндекс передаются только данные с флагом "1"
- `GARANGE_NUMBER` - опционально "человеческий" идентификатор ТС для перевозчика
- `REG_NUMBER` - госномер ТС, есть не всегда
- `ROUTE_DESCR` - текстовый идентификатор траектории маршрута в одном направлении. Нет стандартного формата, можно использовать только как "хэш", как ключ для группировки точек по траекториям

# Создание коннектора к БД

In [2]:
pg = PostgreSQL(
    host=HOST,
    database=DB,
    login=LOGIN,
    password=PASSWORD
)

# Создание списка файлов для импорта в БД

In [3]:
list_filename = ['log/' + file for file in os.listdir('log')]

In [4]:
len(list_filename)

347

# Запись в БД

## Запись логов в БД

In [26]:
%%time 

vehicle_type = ['bus', 'tramway', 'minibus', 'trolleybus']

for file in list_filename:
    parser_ = ParseLog(file)
    df = parser_.transform_df()
    
    for vehicle in vehicle_type:
        df_into = df[df.vehicle_type == vehicle]
        pg.into_pg_table(
            pg_table_name=vehicle,
            df=df_into
        )

CPU times: user 28min 15s, sys: 29.5 s, total: 28min 45s
Wall time: 1h 26min 15s


## Запись логов, которые были записаны в PostgreSQL

In [28]:
list_filename_df = [file for file in os.listdir('log')]

In [31]:
df_file = pd.DataFrame(list_filename_df,columns=['file_name'])

In [33]:
pg.into_pg_table(pg_table_name='written_files',df=df_file)

# Чтение из БД

In [5]:
df = pg.execute_to_df("""select * from bus limit 1""")

df.head()

,timestamp,clid,uuid,route,vehicle_type,latitude,longitude,speed,direction,production,garange_number,reg_number,route_descr
0,2022-05-26 16:27:47,cherepovets,ak1456_2058,#,bus,59.128468,37.837383,24.98,326,0,,е440хк_35 [БВ (Н)],\n


# Создание карты

In [94]:
with open('config.json') as f:
    config = json.load(f)

_map = KeplerGl()
_map.config = config

_map.add_data(data=df_draw, name='yelyi5ow8')

_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['yelyi5ow8'], 'id': '3ynfdw2…